In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import sys
import re
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_clinvar = pd.read_csv('./p04_clinvar_summary.csv', index_col = 0)
df_clinvar = df_clinvar[df_clinvar.total_reads > 200]
df_clinvar.index = [i for i in range(len(df_clinvar))]

In [3]:
df_clinvar.head()

,id,name,target,snp_id,sublibrary,5_homology,spacer_add_g,spacer_length,scaffold,polyT,...,-7,-6,-5,-4,-3,-2,-1,Eff,Max,Indel
0,clinvar_rs121909283_gCaGATGGGGCTCCTGGATCA_1_R1,NM_018055.5(NODAL):c.778G>A (p.Gly260Arg),NODAL,rs121909283,LEG14_Classic_PAM,TGTGGAAAGGACGAAACACC,gCaGATGGGGCTCCTGGATCA,20,GTCATAGTTCCATTAAAGCCAAAAGTGGCTTTGATGTTTCTATGAT...,TTTTTTT,...,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0017,0.0017,0.005814
1,nontarget_control_rs727503277_gGTGTAATCGACTCGA...,NM_000257.4(MYH7):c.632C>T (p.Pro211Leu),MYH7,rs727503277,nontarget_control,TGTGGAAAGGACGAAACACC,gGTGTAATCGACTCGAACTTCGTGT,24,GTCATAGTTCCATTAAAGCCAAAAGTGGCTTTGATGTTTCTATGAT...,TTTTTTT,...,0.0,0.0004,0.0000,0.0000,0.0,0.0,0.0,0.0004,0.0004,0.002784
2,clinvar_rs147025073_gATGTTTGCACCCaTCCTGGAT_1_R2,NM_000264.5(PTCH1):c.3376G>A (p.Val1126Ile),PTCH1,rs147025073,LEG14_Classic_PAM,TGTGGAAAGGACGAAACACC,gATGTTTGCACCCaTCCTGGAT,21,GTCATAGTTCCATTAAAGCCAAAAGTGGCTTTGATGTTTCTATGAT...,TTTTTTT,...,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0016,0.0020,0.015035
3,clinvar_rs145448362_gCATCCTCGTCGaGGGCATTC_1_R3,NM_173477.5(USH1G):c.83C>T (p.Pro28Leu),USH1G,rs145448362,LEG14_Classic_PAM,TGTGGAAAGGACGAAACACC,gCATCCTCGTCGaGGGCATTC,20,GTCATAGTTCCATTAAAGCCAAAAGTGGCTTTGATGTTTCTATGAT...,TTTTTTT,...,0.0,0.0000,0.0000,0.0011,0.0,0.0,0.0,0.2352,0.2352,0.005274
4,nontarget_control_rs61751444_gGCGCCTGAGGATCATG...,NM_001110792.2(MECP2):c.961C>T (p.Arg321Trp),MECP2,rs61751444,nontarget_control,TGTGGAAAGGACGAAACACC,gGCGCCTGAGGATCATGTCGAGCGCC,25,GTCATAGTTCCATTAAAGCCAAAAGTGGCTTTGATGTTTCTATGAT...,TTTTTTT,...,0.0,0.0000,0.0003,0.0000,0.0,0.0,0.0,0.0003,0.0003,0.005212


In [4]:
df_clinvar = df_clinvar[df_clinvar['sample'].isin(['ABE_R1', 'ABE_R2'])].copy()
df_clinvar = df_clinvar[df_clinvar.sublibrary.isin(['LEG14_Classic_PAM', 'positive_control'])]

for idx in df_clinvar.index:
    if df_clinvar.loc[idx, 'sublibrary']  == 'positive_control':
        df_clinvar.loc[idx, 'snp_id'] = 'positive_control'

In [5]:
df_clinvar.columns

Index(['id', 'name', 'target', 'snp_id', 'sublibrary', '5_homology',
       'spacer_add_g', 'spacer_length', 'scaffold', 'polyT', 'bc1',
       'target_seq', 'target_seq_adj', 'bc2', '3_homology', 'full_plasmid',
       'full_plasmid_length', 'sample', 'total_reads', 'Pos', '-25', '-24',
       '-23', '-22', '-21', '-20', '-19', '-18', '-17', '-16', '-15', '-14',
       '-13', '-12', '-11', '-10', '-9', '-8', '-7', '-6', '-5', '-4', '-3',
       '-2', '-1', 'Eff', 'Max', 'Indel'],
      dtype='object')

In [6]:
dct_acgt = {'A':'T', 'T':'A', 'C':'G', 'G':'C'}

lst_df = []

for gd_adg in df_clinvar.spacer_add_g.unique():
    df_clinvar_sub = df_clinvar[df_clinvar.spacer_add_g == gd_adg]
    gd = gd_adg[1:].upper()
    target = df_clinvar_sub.target_seq_adj.to_list()[0].upper()
    if gd in target:
        flag =  1
        st = target.index(gd)
        sp = st + len(gd)
        tg = target[sp-25-6: sp+10]
    else:
        flag = 0
        target = ''.join([dct_acgt[i] for i in target][::-1])
        st = target.index(gd)
        sp = st + len(gd)
        tg = target[sp-25-6: sp+10]
    scaffold = df_clinvar_sub.scaffold.to_list()[0].upper()
    for col in range(-25, 0):
        lst = df_clinvar_sub[str(col)].to_list()
        if np.mean(lst) > 0:
            df_gd = pd.DataFrame()
            df_gd.loc[gd_adg, 'spacer'] = gd
            df_gd.loc[gd_adg, 'target'] = tg
            df_gd.loc[gd_adg, 'scaffold'] = scaffold
            df_gd.loc[gd_adg, 'strand'] = flag
            df_gd.loc[gd_adg, 'spacer_length'] = len(gd)
            df_gd.loc[gd_adg, 'pos'] = col
            df_gd.loc[gd_adg, 'context'] = tg[col - 10 - 3: col - 10 + 4]
            df_gd.loc[gd_adg, 'eff'] = np.mean(lst)
            lst_df.append(df_gd)
            
df_editing = pd.concat(lst_df, ignore_index=True)
    
    

In [7]:
df_editing

,spacer,target,scaffold,strand,spacer_length,pos,context,eff
0,CAGATGGGGCTCCTGGATCA,TTCAACCTGATCAGATGGGGCTCCTGGATCATCTACCCCAA,GTCATAGTTCCATTAAAGCCAAAAGTGGCTTTGATGTTTCTATGAT...,1.0,20.0,-22.0,CTGATCA,0.000033
1,CAGATGGGGCTCCTGGATCA,TTCAACCTGATCAGATGGGGCTCCTGGATCATCTACCCCAA,GTCATAGTTCCATTAAAGCCAAAAGTGGCTTTGATGTTTCTATGAT...,1.0,20.0,-19.0,ATCAGAT,0.002250
2,CAGATGGGGCTCCTGGATCA,TTCAACCTGATCAGATGGGGCTCCTGGATCATCTACCCCAA,GTCATAGTTCCATTAAAGCCAAAAGTGGCTTTGATGTTTCTATGAT...,1.0,20.0,-17.0,CAGATGG,0.002133
3,CAGATGGGGCTCCTGGATCA,TTCAACCTGATCAGATGGGGCTCCTGGATCATCTACCCCAA,GTCATAGTTCCATTAAAGCCAAAAGTGGCTTTGATGTTTCTATGAT...,1.0,20.0,-4.0,TGGATCA,0.000033
4,CATCCTCGTCGAGGGCATTC,GGGGGTCATGCCATCCTCGTCGAGGGCATTCAGCTCCTTTC,GTCATAGTTCCATTAAAGCCAAAAGTGGCTTTGATGTTTCTATGAT...,0.0,20.0,-24.0,GTCATGC,0.000467
...,...,...,...,...,...,...,...,...
7442,CCACAAGCACCCTGAGGACCAGGCG,GGGAAACCACAAGCACCCTGAGGACCAGGCGGACCCCTTTC,GTCATAGTTCCATTAAAGCCAAAAGTGGCTTTGATGTTTCTATGAT...,0.0,25.0,-21.0,CACAAGC,0.168600
7443,CCACAAGCACCCTGAGGACCAGGCG,GGGAAACCACAAGCACCCTGAGGACCAGGCGGACCCCTTTC,GTCATAGTTCCATTAAAGCCAAAAGTGGCTTTGATGTTTCTATGAT...,0.0,25.0,-20.0,ACAAGCA,0.105900
7444,CCACAAGCACCCTGAGGACCAGGCG,GGGAAACCACAAGCACCCTGAGGACCAGGCGGACCCCTTTC,GTCATAGTTCCATTAAAGCCAAAAGTGGCTTTGATGTTTCTATGAT...,0.0,25.0,-17.0,AGCACCC,0.196100
7445,CCACAAGCACCCTGAGGACCAGGCG,GGGAAACCACAAGCACCCTGAGGACCAGGCGGACCCCTTTC,GTCATAGTTCCATTAAAGCCAAAAGTGGCTTTGATGTTTCTATGAT...,0.0,25.0,-11.0,CTGAGGA,0.109800


### DL

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchcam.methods import GradCAM

import RNA

In [ ]:
## Prepare inputs

dct_onehot = {'A': [1, 0, 0, 0],
              'C': [0, 1, 0, 0],
              'G': [0, 0, 1, 0],
              'T': [1, 0, 0, 1],
              'N': [0, 0, 0, 0]}

list_input1 = []

for idx in df_editing.index:
    lst = []
    gd = df_editing.loc[idx, 'spacer'].upper()
    tg = df_editing.loc[idx, 'target'].upper()
    ## target
    lst.append([dct_onehot[i] for i in tg])
    ## spacer
    lst_tg = list(tg)
    st = tg.index(gd)
    sp = st + len(gd)
    for i in range(0, st):
        lst_tg[i] = 'N'
    for i in range(sp, len(lst_tg)):
        lst_tg[i] = 'N'
    lst.append([dct_onehot[i] for i in lst_tg])
    ## editing region
    lst_tg = list(tg)
    st = tg.index(gd)
    sp = st + len(gd)
    pos = df_editing.loc[idx, 'pos']
    st1 = int(pos + sp - 3)
    sp1 = int(pos + sp + 4)
    for i in range(0, st1):
        lst_tg[i] = 'N'
    for i in range(sp1, len(lst_tg)):
        lst_tg[i] = 'N'
    lst.append([dct_onehot[i] for i in lst_tg])
    ## C-region + PAM
    lst_tg = list(tg)
    for i in range(0, len(lst_tg) - 16):
        lst_tg[i] = 'N'
    lst.append([dct_onehot[i] for i in lst_tg])
    ###
    a = np.array(lst, dtype=np.float32)
    t = torch.from_numpy(a)
    list_input1.append(t)
    
input_data1 = torch.stack(list_input1, dim=0)
###
list_input2 = []
for idx in df_editing.index:
    gd = df_editing.loc[idx, 'spacer'].upper()
    scaffold = df_editing.loc[idx, 'scaffold'].upper()
    gd_ext = 'N'*(25 - len(gd)) + gd
    seq = gd_ext + scaffold
    fc = RNA.fold_compound(seq)
    (ss, mfe) = fc.mfe()
    lst = list(RNA.ptable(ss))
    N = lst[0]
    lst = lst[1:]
    a = np.zeros((1, N, N), dtype=np.float32)
    for k, v in enumerate(lst):
        if v > 0:
            a[0, k, v-1] = 1
    t = torch.from_numpy(a)
    list_input2.append(t)
    
input_data2 = torch.stack(list_input2, dim=0)

input_label = torch.Tensor([[i] for i in df_editing.eff.to_list()])

In [ ]:
torch.save(input_data1, 'plot/p15_input1_tensor.pt')
torch.save(input_data2, 'plot/p15_input2_tensor.pt')
torch.save(input_label, 'plot/p15_input_label.pt')

In [11]:
# input1 = torch.load('./p15_input1_tensor.pt')
# input2 = torch.load('./p15_input2_tensor.pt')
# labels = torch.load('./p15_input_label.pt')

In [12]:
# print(input1.size())
# print(input2.size())
# print(labels.size())

torch.Size([7447, 4, 41, 4])
torch.Size([7447, 1, 133, 133])
torch.Size([7447, 1])


In [13]:
# class TwoBranchCNNRegressor(nn.Module):
#     def __init__(self, input_channels1, input_channels2, output_size):
#         super(TwoBranchCNNRegressor, self).__init__()

#         # CNN branch for the first input
#         self.branch1 = nn.Sequential(
#             nn.Conv2d(in_channels=input_channels1, out_channels=32, kernel_size=4, padding=(4, 4)),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=4),
#             nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, padding=(4, 4)),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=4),
#             nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, padding=(4, 4)),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=4),
#             nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=4)
#         )

#         # CNN branch for the second input
#         self.branch2 = nn.Sequential(
#             nn.Conv2d(in_channels=input_channels2, out_channels=32, kernel_size=15),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2)
#         )

#         # Fully connected layer for merging features
#         self.fc = nn.Linear(205600, 111648, 64)
        
#         # Regression output layer
#         self.output_layer = nn.Linear(64, output_size)

#     def forward(self, x1, x2):
#         # Process the first input through the first CNN branch
#         x1 = self.branch1(x1)
#         # Process the second input through the second CNN branch
#         x2 = self.branch2(x2)
        
#         # Reshape the output to be compatible for concatenation
#         x1 = x1.view(x1.size(0), -1)
#         x2 = x2.view(x2.size(0), -1)
        
#         # Dynamically calculate the size of the linear layer input again
#         self.fc_input_size = x1.size(1) + x2.size(1)

#         # Concatenate the features from both branches
#         x = torch.cat((x1, x2), dim=1)
        
#         print(x1.size())
#         print(x2.size())
#         print(x.size())
#         # Fully connected layer
#         x = self.fc(x)
        
#         # Regression output layer
#         output = self.output_layer(x)
        
#         return output

#     def calculate_fc_input_size(self, in_channels1, height1, width1, in_channels2, height2, width2):
#         # Dummy input data to get the output shape after convolutional layers
#         input_data1 = torch.randn(1, in_channels1, height1, width1)
#         input_data2 = torch.randn(1, in_channels2, height2, width2)

#         # Forward pass to calculate the output shape after convolutional layers
#         output1 = self.branch1(input_data1)
#         output2 = self.branch2(input_data2)

#         # Concatenate and flatten the features
#         concatenated_features = torch.cat((output1, output2), dim=1)
#         flattened_size = concatenated_features.view(1, -1).size(1)

#         return flattened_size



In [14]:
# # Example usage
# input_channels1, height1, width1 = 4, 41, 4  # Replace with your actual dimensions
# input_channels2, height2, width2 = 1, 133, 133  # Replace with your actual dimensions
# output_size = 1  # For regression, set the output size based on your task

# # Create an instance of the model
# model = TwoBranchCNNRegressor(input_channels1, input_channels2, output_size)

# # Define your loss function and optimizer
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Forward pass
# output = model(input1, input2)

# # Dummy target data
# target = labels  # Replace with your actual target values

# # Compute the loss
# loss = criterion(output, target)

# # Backward pass and optimization step
# optimizer.zero_grad()
# loss.backward()
# optimizer.step()


RuntimeError: [enforce fail at alloc_cpu.cpp:117] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 91819315200 bytes. Error code 12 (Cannot allocate memory)

In [42]:
# type(input_data1)

torch.Tensor

In [43]:
# input_data1.size()

torch.Size([32, 3, 32, 32])

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision
# import torchvision.transforms as transforms
# from torch.utils.data import DataLoader

# # Set device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Define a simple CNN model
# class SimpleCNN(nn.Module):
#     def __init__(self):
#         super(SimpleCNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
#         self.relu = nn.ReLU()
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
#         self.fc1 = nn.Linear(32 * 8 * 8, 256)
#         self.fc2 = nn.Linear(256, 10)

#     def forward(self, x):
#         x = self.pool(self.relu(self.conv1(x)))
#         x = self.pool(self.relu(self.conv2(x)))
#         x = x.view(-1, 32 * 8 * 8)
#         x = self.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

# # Transformations for data augmentation
# transform_train = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomCrop(32, padding=4),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
# ])

# transform_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
# ])

# # Download and load CIFAR-10 dataset
# train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
# test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# # DataLoader
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

# # Instantiate the model and move it to the device
# model = SimpleCNN().to(device)

# # Loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Training loop
# num_epochs = 5

# for epoch in range(num_epochs):
#     model.train()  # Set the model to training mode
#     total_loss = 0.0

#     for i, (inputs, labels) in enumerate(train_loader):
#         inputs, labels = inputs.to(device), labels.to(device)

#         # Forward pass
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)

#         # Backward pass and optimization step
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     average_loss = total_loss / len(train_loader)
#     print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}")

# # Testing loop
# model.eval()  # Set the model to evaluation mode
# correct = 0
# total = 0

# with torch.no_grad():
#     for inputs, labels in test_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         outputs = model(inputs)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# accuracy = correct / total
# print(f"Accuracy on the test set: {100 * accuracy:.2f}%")


In [13]:
# from Bio.SeqUtils import MeltingTemp as mt
# from Bio.Seq import Seq

# def generate_feature_mx(df_ed):
#     df_stat_feat = pd.DataFrame()

#     lst_k2 = [i+j for i in ['A', 'C', 'G', 'T'] for j in ['A', 'C', 'G', 'T']]

#     for idx0 in df_ed.index:
#         ##
#         idx = len(df_stat_feat)
#         df_stat_feat.loc[idx, 'eff'] = df_ed.loc[idx0, 'eff']
#         df_stat_feat.loc[idx, 'SGN_Strand'] = df_ed.loc[idx0, 'strand']
#         df_stat_feat.loc[idx, 'SGN_Length'] = df_ed.loc[idx0, 'spacer_length']/25
#         ## feature
#         seq_nb = df_ed.loc[idx0, 'context']
#         df_stat_feat.loc[idx, 'Editing_Position'] = -1 * df_ed.loc[idx0, 'pos']
#         for k,v in enumerate(seq_nb):
#             if k == 3:
#                 continue
#             for bs in ['A', 'C', 'G', 'T']:
#                 col = 'Editing_P'+str(k - 3)+'_'+bs
#                 if bs == v:
#                     df_stat_feat.loc[idx, col] = 1
#                 else:
#                     df_stat_feat.loc[idx, col] = 0
#         #
#         seq_nb2 = Seq(seq_nb)
#         mt_w = mt.Tm_Wallace(seq_nb)
#         df_stat_feat.loc[idx, 'Editing_mt'] = mt_w/100
#         #
#         sgn = df_ed.loc[idx0, 'target']
#         for k in range(-1*len(sgn),0):
#             v = sgn[k]
#             for bs in ['A', 'C', 'G', 'T']:
#                 l = k+10
#                 if l >= 0:
#                     l = l + 1
#                 col = 'SGN_P'+str(l)+'_'+bs
#                 if bs == v:
#                     df_stat_feat.loc[idx, col] = 1
#                 else:
#                     df_stat_feat.loc[idx, col] = 0
#         for k in range(0, 30, 4):
#             seq_nb2 = Seq(sgn[k:k+10])
#             mt_w = mt.Tm_Wallace(seq_nb)
#             df_stat_feat.loc[idx, 'SGN_mt_w_'+str(k)] = mt_w/100
#     return df_stat_feat
